In [2]:
import os

import numpy as np
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM, Wav2Vec2Processor

models = [
    "classla/wav2vec2-xls-r-parlaspeech-hr-lm",
    "classla/wav2vec2-xls-r-parlaspeech-hr",
    "classla/wav2vec2-large-slavic-parlaspeech-hr",
    "classla/wav2vec2-large-slavic-parlaspeech-hr-lm",
]
results = list()
for model in models:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # load model and tokenizer
    if model.endswith("-lm"):
        processor = Wav2Vec2ProcessorWithLM.from_pretrained(model,)
    else:
        processor = Wav2Vec2Processor.from_pretrained(model, pad_token="[PAD]")
    model = Wav2Vec2ForCTC.from_pretrained(model)
    # read the wav file
    speech, sample_rate = sf.read("audio.wav")

    overlap_seconds = 1

    indices = np.arange(
        0, speech.shape[0], 10 * 60 * sample_rate, dtype=int
    ).tolist() + [-1]

    transcripts = list()
    for start, stop in zip(indices[0:-2], indices[1:]):
        # If overlap would make the segment go
        # over the end, correct stop variable:
        if stop + overlap_seconds * sample_rate >= speech.shape[0]:
            stop = -1
        speech_segment = speech[start:stop]
        inputs = processor(
            speech_segment, sampling_rate=sample_rate, return_tensors="pt"
        )
        with torch.no_grad():
            logits = model(**inputs).logits
        try:
            transcription = processor.batch_decode(logits.numpy()).text[0]
        except:
            prediction_ids = torch.argmax(logits, dim=-1)
            transcription = processor.batch_decode(prediction_ids)

        transcripts.append(transcription)
    results.append({"model": model, "transcription": " ".join(transcripts)})

import pandas as pd

pd.DataFrame(data=results).to_csv("003_all_models_transcriptions.csv", index=False)


Downloading:   0%|          | 0.00/328 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/868M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

Only 0 unigrams passed as vocabulary. Is this small or artificial data?


In [13]:
logits.numpy().shape

(1, 29999, 50)